# TF-IDF

In [1]:
import pandas as pd
import numpy as np

from pandas.errors import EmptyDataError

# 정규식 활용 -> 특수문자 제거
import re

# 한글 형태소 분석기
from konlpy.tag import Okt
okt = Okt()

# keybert
from keybert import KeyBERT

In [2]:
date_str = '20250820'

In [3]:
def safe_read_csv(path):
    try:
        return pd.read_csv(path)
    except EmptyDataError:
        print(f"⚠️ {path} 는 비어있음 → 스킵")
        return pd.DataFrame()   # 빈 DF 반환
    except FileNotFoundError:
        print(f"❌ {path} 파일 없음 → 스킵")
        return pd.DataFrame()

# 파일 불러오기
raw_data_i = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_i_{date_str}.csv")
raw_data_g = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_g_{date_str}.csv")
raw_data_gen_1 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_00000-19999.csv")
raw_data_gen_2 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_20000-39999.csv")
raw_data_gen_3 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_40000-59999.csv")
raw_data_gen_4 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_60000-79999.csv")
raw_data_gen_5 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_80000-99999.csv")

# 하나의 파일로 유니온 (비어있으면 자동 무시)
raw_data = pd.concat([
    raw_data_gen_1, raw_data_gen_2, raw_data_gen_3,
    raw_data_gen_4, raw_data_gen_5,
    raw_data_i, raw_data_g
], ignore_index=True)

print("총 데이터 크기:", raw_data.shape)

# 완전일치 중복기사 제거
df = raw_data.drop_duplicates(subset=['title', 'text'], keep='first').copy()

# title에 [속보] 또는 [포토] 포함 시 text를 NaN으로 변경
df.loc[df['title'].str.contains(r'\[속보\]|\[포토\]', regex=True), 'text'] = np.nan

# null셀을 빈칸으로 만들기
df['title'] = df['title'].fillna('')
df['text'] = df['text'].fillna('')

# 특수문자 등 전처리
df['text_clean'] = df['text'].apply(lambda t: re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', t))
df['title_clean'] = df['title'].apply(lambda t: re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', t))
df.head(3)

⚠️ /Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_20250820_20000-39999.csv 는 비어있음 → 스킵
⚠️ /Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_20250820_40000-59999.csv 는 비어있음 → 스킵
⚠️ /Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_20250820_60000-79999.csv 는 비어있음 → 스킵
총 데이터 크기: (245, 4)


,url,title,text,publish_date,text_clean,title_clean
0,https://www.hankyung.com/article/2025082000057,"67세 마돈나, '38살 연하' 연인과 이탈리아서 화려한 생일파티","마돈나, 자메이카 출신 29세 연인 모리스와 생일 보내\n\n출처=마돈나 인스타그램...",2025-08-20 09:14:39+09:00,마돈나 자메이카 출신 세 연인 모리스와 생일 보내 출처 마돈나 인스타그램 ...,세 마돈나 살 연하 연인과 이탈리아서 화려한 생일파티
1,https://www.hankyung.com/article/2025082000044,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaN,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다 회담은 뉴욕 증시가 마감할 무렵 ...,한국경제
2,https://www.hankyung.com/article/2025082000107,"안철수, 복당 1순위 '이준석' 선택 유일…李 ""옳고 강한 분""",'복당 1순위 홍준표? 이준석?' 질문에\n\n당권 주자 중 혼자 '이준석' 꼽은 ...,2025-08-20 09:39:32+09:00,복당 순위 홍준표 이준석 질문에 당권 주자 중 혼자 이준석 꼽은 안철...,안철수 복당 순위 이준석 선택 유일 옳고 강한 분


In [4]:
# 불용어제거 -> 형태소 조사 삭제 -> tf-idf -> 가장 중요한 10개 단어 뽑아내기

In [5]:
# 불용어 불러오기 (줄바꿈 기준 분리)
with open("/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/stopwords_kor.txt", "r", encoding="utf-8") as f:
    stopwords = f.read()

# 불용어 제거 함수
def remove_stopwords(text_clean):
    if not isinstance(text_clean, str):
        return ''
    return ' '.join([word for word in text_clean.split() if word not in stopwords])

# 조사 제거 함수
def remove_josa(text):
    if not isinstance(text, str):
        return ''
    return ' '.join([word for word, pos in okt.pos(text) if pos != 'Josa'])

# 조사 제거 → 불용어 제거 순차 적용
df['title_clean'] = df['title_clean'].apply(remove_josa)  # 조사 제거
df['title_clean'] = df['title_clean'].apply(remove_stopwords)  # 불용어 제거

df['text_clean'] = df['text_clean'].apply(remove_josa)  # 조사 제거
df['text_clean'] = df['text_clean'].apply(remove_stopwords)  # 불용어 제거

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1) 코퍼스 준비 (NaN 방지)
corpus = df['text_clean'].fillna('')

# 2) 벡터라이저 설정
# - tokenizer=lambda s: s.split(): 이미 공백으로 토큰화된 텍스트를 그대로 사용
# - lowercase=False: 한글/대소문 유지
# - ngram_range=(1,1): 단어 단위(필요 시 (1,2) 등으로 확장)
# - min_df, max_df, max_features는 데이터 규모에 맞게 조절
vectorizer = TfidfVectorizer(
    tokenizer=lambda s: s.split(),
    token_pattern=None,
    lowercase=False,
    ngram_range=(1,1),
    min_df=2,           # 2개 미만 문서에만 나온 단어 제거
    max_df=0.7,         # 70% 넘는 문서에 등장하는 흔한 단어 제거
    max_features=8000   # 상위 8천개 단어만 사용 (메모리 절약용)
)

# 3) 학습 및 변환 (희소행렬)
X = vectorizer.fit_transform(corpus)   # shape: (문서수, 단어수)
terms = vectorizer.get_feature_names_out()

# 4) (옵션) DataFrame으로 시각화 — 메모리 주의
# tfidf_df = pd.DataFrame.sparse.from_spmatrix(X, columns=terms)

# 5) 각 문서별 TF-IDF 상위 키워드 뽑기 함수
def top_tfidf_terms(row_index: int, topk: int = 10):
    row = X.getrow(row_index).toarray().ravel()
    if not np.any(row):
        return []
    idx = np.argsort(row)[::-1][:topk]
    return list(zip(terms[idx], row[idx]))

# 6) 모든 문서에 대해 상위 키워드 뽑아 df에 붙이기 (리스트→문자열)
topk = 10
df['tfidf_top_terms'] = [
    ', '.join([f'{t}:{w:.3f}' for t, w in top_tfidf_terms(i, topk)]) 
    for i in range(X.shape[0])
]

In [7]:
df.head()

,url,title,text,publish_date,text_clean,title_clean,tfidf_top_terms
0,https://www.hankyung.com/article/2025082000057,"67세 마돈나, '38살 연하' 연인과 이탈리아서 화려한 생일파티","마돈나, 자메이카 출신 29세 연인 모리스와 생일 보내\n\n출처=마돈나 인스타그램...",2025-08-20 09:14:39+09:00,마돈나 자메이카 출신 연인 모리스 생일 출처 마돈나 인스타그램 팝스타 마돈나 생일 ...,마돈나 연하 연인 이탈리아 화려한 생일 파티,"생일:0.484, 연인:0.290, 파티:0.290, 남자친구:0.193, 인스타그..."
1,https://www.hankyung.com/article/2025082000044,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaN,트럼프 대통령 푸틴 러시아 대통령 만났습니다 회담 뉴욕 증시 마감 관망 나타났습니다...,한국,"관세:0.523, 금리:0.254, 물가:0.211, 수입:0.191, 파월:0.1..."
2,https://www.hankyung.com/article/2025082000107,"안철수, 복당 1순위 '이준석' 선택 유일…李 ""옳고 강한 분""",'복당 1순위 홍준표? 이준석?' 질문에\n\n당권 주자 중 혼자 '이준석' 꼽은 ...,2025-08-20 09:39:32+09:00,복당 순위 홍준표 이준석 질문 당권 주자 이준석 꼽 안철수 이준석 옳고 강해 DAS...,안철수 복당 순위 이준석 선택 유일 옳고 강한,"후보:0.555, 안철수:0.360, 국민:0.322, 의원:0.202, 홍준표:0..."
3,https://www.hankyung.com/article/2025082000156,"코스피, 장 초반 2%대로 낙폭 확대…시총 상위주 일제히 '파란불'",20일 서울 중구 하나은행 본점 딜링룸 스크린에 지수가 표시돼 있다. /사진=연합뉴...,2025-08-20 09:24:05+09:00,서울 중구 하나은행 본점 딜링룸 스크린 지수 표시 연합뉴스 코스피지수 하락 세로 출...,코스피 초반 낙폭 확대 위주 일제 파란,"우위:0.309, 내린:0.248, 지수:0.242, 낙폭:0.212, 원전:0.2..."
4,https://www.hankyung.com/article/2025082000167,"[속보] 특검, 김건희 구속기간 이달 31일까지로 연장",,2025-08-20 08:59:57+09:00,,속보 특검 김건희 구속 기간 연장,


In [8]:
from collections import Counter
import re

# 1. 전체 키워드 모으기 (점수 제거)
all_terms = []
for row in df['tfidf_top_terms']:
    if not isinstance(row, str):
        continue
    # "단어:점수" → 단어만 추출
    terms = [re.split(r':', t)[0] for t in row.split(', ')]
    all_terms.extend(terms)

# 2. 빈도수 계산
term_freq = Counter(all_terms)

# 3. DataFrame으로 변환
term_freq_df = pd.DataFrame(term_freq.items(), columns=['term', 'count'])
term_freq_df = term_freq_df.sort_values(by='count', ascending=False).reset_index(drop=True)

term_freq_df.head(10)

,term,count
0,,15
1,미국,12
2,국민,9
3,중국,8
4,장관,7
5,사업,7
6,대통령,6
7,AI,6
8,가격,6
9,경찰,6


In [9]:
term_freq_df.to_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/text_TFIDF.csv", index=False)